In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

#Installing tensorflow 2.0
#!pip install -q tensorflow==2.0.0-alpha0

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
titanic_train = pd.read_csv('/kaggle/input/train.csv')
titanic_test = pd.read_csv('/kaggle/input/test.csv')

titanic_train.head()

In [ ]:
titanic_test.head()

In [ ]:
titanic_train.info()

In [ ]:
titanic_train.describe()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(titanic_train, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_train['Embarked'].value_counts()

In [ ]:
embarked_replace = 'S'
Y_train = X_train['Survived']
Y_val = X_val['Survived']

dropping_columns = ['Survived', 'Name', 'Ticket', 'Cabin']
X_train = X_train.drop(dropping_columns, axis=1)
X_val = X_val.drop(dropping_columns, axis=1)

In [ ]:
X_train['Embarked'] = X_train['Embarked'].fillna(embarked_replace)
X_val['Embarked'] = X_val['Embarked'].fillna(embarked_replace)

In [ ]:
X_train.head()

In [ ]:
X_val.head()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [ ]:
num_pipeline = Pipeline([        
                          ('imputer', SimpleImputer(strategy="mean")),        
                          ('std_scaler', StandardScaler()),    
                        ])

#embarked_pipeline = Pipeline([        
#                          ('encoder', LabelEncoder()),    
#                        ])

In [ ]:
num_attribs = ['Pclass','Age','SibSp','Parch','Fare']
#embarked_attrib = ['Embarked']
cat_attribs = ['Sex','Embarked']

In [ ]:
full_pipeline = ColumnTransformer([        
    ("num", num_pipeline, num_attribs),        
 #  ("e_cat", embarked_pipeline, embarked_attrib),   
    ("cat", OneHotEncoder(), cat_attribs)   
])

In [ ]:
X_train_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_val_prepared = full_pipeline.transform(X_val)

In [ ]:
X_train_prepared.shape

In [ ]:
X_train_prepared

In [ ]:
Y_val.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential([       
                                     keras.layers.Dense(15, activation="relu", input_dim=10),    
                                     keras.layers.Dense(10, activation="relu"),    
                                     keras.layers.Dense(1, activation="sigmoid"),
                                ])

In [ ]:
model.summary()

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
model.get_layer('dense_1') is hidden1

In [ ]:
model.compile(loss="binary_crossentropy",              
              optimizer="sgd",              
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train_prepared, Y_train, epochs=1,
                    validation_data=(X_val_prepared, Y_val))

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
#save_fig("keras_learning_curves_plot")
plt.show()

**Preparing TEST data**

In [ ]:
test_dropping_columns = ['Name', 'Ticket', 'Cabin']

X_test =  titanic_test.drop(test_dropping_columns, axis=1)
X_test['Embarked'] = X_test['Embarked'].fillna(embarked_replace)

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
Y_test_predict_DL = model.predict_classes(X_test_prepared)

SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier

sgd_clf = RandomForestClassifier(random_state=42)
sgd_clf.fit(X_train_prepared, Y_train)

Y_train_pred = sgd_clf.predict(X_train_prepared)

# calculating the mean squared error
mse = np.mean((Y_train - Y_train_pred)**2)
print("MSE :", mse)

# calculating the root mean squared error
rmse = np.sqrt(mse)
print("RMSE :", rmse)

#calculating the r2 score
r2 = r2_score(Y_train, Y_train_pred)
print("r2_score :", r2)


In [ ]:
Y_test_predict_SGD = sgd_clf.predict(X_test_prepared)

In [ ]:
Y_test_predict_SGD

In [ ]:
#Writing to File
submission=pd.DataFrame(titanic_test.loc[:,['PassengerId']])
submission['Survived']=Y_test_predict_SGD
#Any files you save will be available in the output tab below

submission.to_csv('submission.csv', index=False)